# Work in Progress/Add in Future:

- Scrape data for min. 1 week
- Change cities

In [2]:
import os
import sys
from datetime import datetime as dt
from datetime import date
from datetime import timedelta
import glob
import numpy as np
import pandas as pd
sys.path.insert(0, os.path.abspath('..\modules'))
import pricemate as pm
todaydate = date.today().strftime('%Y%m%d')

In [24]:
# scrape data using pricemate module
df_today = pm.multiple_days_df(0,20)


Looking for [chromedriver 79.0.3945.36 win32] driver in cache 
File found in cache by path [C:\Users\Zenbook\.wdm\drivers\chromedriver\79.0.3945.36\win32\chromedriver.exe]


In [28]:
df_today.head()

,arrival_date,arrival_time,depart_date,depart_time,price,seats,tclass,title
0,2020-01-13,01:27,2020-01-12,22:05,110000,99,ekonomi (Subclass S),argo parahyangan 50
1,2020-01-13,02:05,2020-01-12,22:55,110000,99,ekonomi (Subclass S),argo parahyangan 52
2,2020-01-13,02:40,2020-01-12,23:30,110000,99,ekonomi (Subclass S),argo parahyangan 70F
3,2020-01-12,20:29,2020-01-12,17:10,130000,99,ekonomi (Subclass P),mutiara selatan 104
4,2020-01-12,20:59,2020-01-12,17:45,150000,1,eksekutif (Subclass J),argo parahyangan 46


In [29]:
df_today['scrape_date'] = dt.date(dt.now())

# export scrape output to new csv
df_today.to_csv('..\scrape-csv\gmr-bdg-' + todaydate + '.csv', index=False)

In [30]:
df_today.head()

,arrival_date,arrival_time,depart_date,depart_time,price,seats,tclass,title,scrape_date
0,2020-01-13,01:27,2020-01-12,22:05,110000,99,ekonomi (Subclass S),argo parahyangan 50,2020-01-12
1,2020-01-13,02:05,2020-01-12,22:55,110000,99,ekonomi (Subclass S),argo parahyangan 52,2020-01-12
2,2020-01-13,02:40,2020-01-12,23:30,110000,99,ekonomi (Subclass S),argo parahyangan 70F,2020-01-12
3,2020-01-12,20:29,2020-01-12,17:10,130000,99,ekonomi (Subclass P),mutiara selatan 104,2020-01-12
4,2020-01-12,20:59,2020-01-12,17:45,150000,1,eksekutif (Subclass J),argo parahyangan 46,2020-01-12


In [69]:
# import and combine past scrape data with today's scrape, if any
df = df_today
files = glob.glob('..\scrape-csv\*.csv')
for file in files:
    scrapedate = file[-12:-4]
    if scrapedate < todaydate:
        past_df = pd.read_csv(file)
        past_df['scrape_date'] = pd.to_datetime(scrapedate)
        df = df.append(past_df)


# print('Earliest scrape: ' + str(min(df['scrape_datetime'])))
# print('Latest scrape: ' + str(max(df['scrape_datetime'])))

In [68]:
# change or remove columns
"""
for file in files:
    past_df = pd.read_csv(file)
    if 'arrive_time' in past_df.columns:
        past_df['arrival_time'] = past_df['arrive_time']
        past_df = past_df.drop(columns = ['arrive_time'])
        past_df.to_csv(file, index=False)
"""

In [70]:
# duplicate combined dataframe
df_raw = df
df.head()

,arrival_date,arrival_datetime,arrival_time,depart_date,depart_datetime,depart_time,price,scrape_date,seats,tclass,title
0,2020-01-13,NaN,01:27,2020-01-12,NaN,22:05,110000,2020-01-12,99,ekonomi (Subclass S),argo parahyangan 50
1,2020-01-13,NaN,02:05,2020-01-12,NaN,22:55,110000,2020-01-12,99,ekonomi (Subclass S),argo parahyangan 52
2,2020-01-13,NaN,02:40,2020-01-12,NaN,23:30,110000,2020-01-12,99,ekonomi (Subclass S),argo parahyangan 70F
3,2020-01-12,NaN,20:29,2020-01-12,NaN,17:10,130000,2020-01-12,99,ekonomi (Subclass P),mutiara selatan 104
4,2020-01-12,NaN,20:59,2020-01-12,NaN,17:45,150000,2020-01-12,1,eksekutif (Subclass J),argo parahyangan 46


## Data Preprocessing
- Add preprocessing here based on diagnostic testing

In [71]:
# concat date and time, then convert to datetime
df['arrival_datetime'] = pd.to_datetime(df['arrival_date'] + ' ' + df['arrival_time'])
df['depart_datetime'] = pd.to_datetime(df['depart_date'] + ' ' + df['depart_time'])

In [35]:
def remove_nonnumeric(string):
    if isinstance(string, str):
        string = ''.join(filter(lambda x: x.isdigit(), string))
    return string

In [72]:
df['price'] = df['price'].apply(remove_nonnumeric).apply(pd.to_numeric)
df['seats'] = df['seats'].apply(remove_nonnumeric).apply(pd.to_numeric)

In [106]:
three_dates = ['arrival_date', 'depart_date', 'scrape_date']
df[three_dates] = df[three_dates].apply(pd.to_datetime)

In [ ]:
# does arrival and depart time need to be converted into dt.time?
# df['arrival_time'] = pd.to_datetime(df['arrival_time']).dt.time
# df['depart_time'] = pd.to_datetime(df['depart_time']).dt.time

## Diagnostic assumptions/test suite
- Arrival date >= depart date
- depart date >= scrape date
- 0 <= seats <= 99


In [109]:
assert min(df['arrival_datetime'] - df['depart_datetime']) > timedelta(0), 'arrival before departure exists'
assert min(df['depart_date'] - df['scrape_date']) >= timedelta(0), 'departure before scrape exists'
assert min(df['seats']) >= 0, 'negative seats exist'
assert max(df['seats']) <= 99, '>99 seats exist'

## View basics of data

In [110]:
df.head()

,arrival_date,arrival_datetime,arrival_time,depart_date,depart_datetime,depart_time,price,scrape_date,seats,tclass,title
0,2020-01-13,2020-01-13 01:27:00,01:27:00,2020-01-12,2020-01-12 22:05:00,22:05:00,110000.0,2020-01-12,99,ekonomi (Subclass S),argo parahyangan 50
1,2020-01-13,2020-01-13 02:05:00,02:05:00,2020-01-12,2020-01-12 22:55:00,22:55:00,110000.0,2020-01-12,99,ekonomi (Subclass S),argo parahyangan 52
2,2020-01-13,2020-01-13 02:40:00,02:40:00,2020-01-12,2020-01-12 23:30:00,23:30:00,110000.0,2020-01-12,99,ekonomi (Subclass S),argo parahyangan 70F
3,2020-01-12,2020-01-12 20:29:00,20:29:00,2020-01-12,2020-01-12 17:10:00,17:10:00,130000.0,2020-01-12,99,ekonomi (Subclass P),mutiara selatan 104
4,2020-01-12,2020-01-12 20:59:00,20:59:00,2020-01-12,2020-01-12 17:45:00,17:45:00,150000.0,2020-01-12,1,eksekutif (Subclass J),argo parahyangan 46


In [111]:
df.describe()

,price,seats
count,3587.000000,9524.000000
mean,167930.025091,35.542629
std,66571.494841,47.391105
min,110000.000000,0.000000
25%,110000.000000,0.000000
50%,150000.000000,0.000000
75%,170000.000000,99.000000
max,375000.000000,99.000000


In [112]:
df.dtypes
# all data types seem to be objects

arrival_date        datetime64[ns]
arrival_datetime    datetime64[ns]
arrival_time                object
depart_date         datetime64[ns]
depart_datetime     datetime64[ns]
depart_time                 object
price                      float64
scrape_date         datetime64[ns]
seats                        int64
tclass                      object
title                       object
dtype: object

## Exploratory questions & assumptions
1. which times, days, classes are the least available?
2. When do the least available tickets start becoming unavailable?

Target variable: % unavailable tickets on a day/time/class 
= (Seats == 0) / (# seats in group)

In [ ]:
# availability by day
